In [ ]:
# THIS ONE HAS ANOTHER SONG_URL THAT ACCOUNTS FOR THE ARTIST NAME WITH "THE" and "AND"! IT IS THE ONE I CURRENTLY USE.
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# SCRAPE SONG NAMES
allthesongs = pd.DataFrame(columns=["Rank", "Song", "Artist", "Year", "Lyrics", "Source"])

for i in range(1964,2024):
    URL = f"http://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_{i}"
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, 'html.parser')

    rows = soup.select('table.wikitable.sortable tr')[1:]

    billboard_data = []
    for row in rows:
        columns = row.find_all(['td', 'th'])
        #HERE, I changeed strip= true to strip = false
        billboard_data.append([col.get_text(strip=False) for col in columns])

    year_data = pd.DataFrame(billboard_data, columns=["Rank", "Song", "Artist"])
    year_data["Year"] = i
    allthesongs = pd.concat([allthesongs, year_data], ignore_index=True)

print(allthesongs["Artist"][71])

allthesongs["Song"] = allthesongs["Song"].str.replace('\"', '').str.lower().str.replace("[^a-zA-Z0-9 ']", '')
#allthesongs["Artist"] = allthesongs["Artist"].str.lower().str.replace(' & ', ' and ')
#allthesongs["Artist"] = allthesongs["Artist"].str.lower().str.replace('&', ' and ')
allthesongs["Artist"] = allthesongs["Artist"].str.lower().str.replace("[^a-zA-Z0-9 ? & ' -]", '')
allthesongs["Artist"] = allthesongs["Artist"].str.replace("'e", "e").str.replace("'o", "o")
allthesongs["Artist"] = allthesongs["Artist"].str.lower().str.replace('featuring', ' feat ')

#THIS CAN MAYBE BE DELETED

#print(allthesongs["Artist"][71])


# SCRAPE THE LYRICS
for s in range(len(allthesongs["Song"])):
    lyrics = "Not set yet."
    results = 12  # arbitrary number

    pre_artist = allthesongs["Artist"].iloc[s]
    if pre_artist == None:
      pre_artist = allthesongs["Artist"][s-1]
      print("DONE")
    #artist = re.split(" featuring | feat | feat. | with | duet | and ", allthesongs["Artist"].iloc[s])[0]
    #artist = re.split(" featuring | feat | feat. | with | duet | and ", pre_artist)[0]
    artist = re.sub(" and ", " & ", pre_artist)
    artist = re.sub("featuring", " feat ", artist)
    #artist = re.sub("and ", "& ", pre_artist)
    #THIS IS THE LINE TO DELETE:
    pre_artist = artist
    artist2 = re.sub("the ", "", artist)
    first_letter = artist2[0]

    #print(artist)
    #print(pre_artist)
    #metro_url = f"http://metrolyrics.com/{allthesongs['Song'].iloc[s]}-lyrics-{artist2}.html"
    song_url2 = f"http://songlyrics.com/{artist}/{allthesongs['Song'].iloc[s]}-lyrics"
    song_url3 = f"http://songlyrics.com/{pre_artist}/{allthesongs['Song'].iloc[s]}-lyrics"
    song_url = f"http://songlyrics.com/{artist2}/{allthesongs['Song'].iloc[s]}-lyrics"
    mode_url = f"http://www.lyricsmode.com/lyrics/{first_letter}/{artist2}/{allthesongs['Song'].iloc[s]}.html"

    urls = [song_url3, song_url2, song_url, mode_url]
    lyric_locs = ['p#songLyricsDiv', 'p#songLyricsDiv', 'p#songLyricsDiv', 'p#lyrics_text']

    for b, URL in enumerate(urls):
        allthesongs.at[s, "Lyrics"] = "Not set yet."
        results = 12  # arbitrary number

        if b != 3:
            URL = URL.replace(" ", "-").lower()

        try:
            response = requests.get(URL)
            soup = BeautifulSoup(response.text, 'lxml')
            lyrics = soup.select_one(lyric_locs[b]).get_text()
        except Exception as e:
            print(f"{s} failed")
        else:
            if lyrics:
                allthesongs.at[s, "Lyrics"] = lyrics
                print(f"{s} success")
                print(URL)
                allthesongs.at[s, "Source"] = b
                break

#allthesongs["Lyrics"] = allthesongs["Lyrics"].replace(to_replace=["\\\n", "\\\t"], value=" ", regex=True)
#allthesongs["Lyrics"] = allthesongs["Lyrics"].str.lower().replace('[^a-zA-Z0-9]', '')
#allthesongs["Lyrics"] = allthesongs["Lyrics"].replace('?', '')
#allthesongs["Lyrics"] = allthesongs["Lyrics"].replace('!', '')
allthesongs["Lyrics"] = allthesongs["Lyrics"].str.lower()
##ADD A PERIOD HERE
missing = round((allthesongs["Lyrics"] == "not set yet").sum() / len(allthesongs) * 100, 2)
allthesongs["Lyrics"] = allthesongs["Lyrics"].replace({"not set yet": "NA",
                                                       "we are not in a position to display these lyrics due to licensing restrictions sorry for the inconvenience": "NA"})

# Set working directory and write to CSV if needed
allthesongs.to_csv("billboard_lyrics_1964-2015.csv", index=False)
allthesongs = pd.read_csv("billboard_lyrics_1964-2015.csv")


Dionne Warwick

dionne warwick


<ipython-input-36-46d2a043a85d>:29: FutureWarning: The default value of regex will change from True to False in a future version.
  allthesongs["Song"] = allthesongs["Song"].str.replace('\"', '').str.lower().str.replace("[^a-zA-Z0-9 ']", '')
<ipython-input-36-46d2a043a85d>:32: FutureWarning: The default value of regex will change from True to False in a future version.
  allthesongs["Artist"] = allthesongs["Artist"].str.lower().str.replace("[^a-zA-Z0-9 ? & ' -]", '')


Streaming output truncated to the last 5000 lines.
3510 success
http://songlyrics.com/ricky-martin/livin'-la-vida-loca-lyrics
3511 success
http://songlyrics.com/702/where-my-girls-at-lyrics
3512 success
http://songlyrics.com/jennifer-lopez/if-you-had-my-love-lyrics
3513 success
http://songlyrics.com/goo-goo-dolls/slide-lyrics
3514 success
http://songlyrics.com/brandy/have-you-ever-lyrics
3515 success
http://songlyrics.com/backstreet-boys/i-want-it-that-way-lyrics
3516 success
http://songlyrics.com/r-kelly-&-celine-dion/i'm-your-angel-lyrics
3517 success
http://songlyrics.com/smash-mouth/all-star-lyrics
3518 success
http://songlyrics.com/sarah-mclachlan/angel-lyrics
3519 success
http://songlyrics.com/santana--feat--rob-thomas/smooth-lyrics
3520 success
http://songlyrics.com/tlc/unpretty-lyrics
3521 success
http://songlyrics.com/destiny's-child/bills-bills-bills-lyrics
3522 success
http://songlyrics.com/eagle-eye-cherry/save-tonight-lyrics
3523 success
http://songlyrics.com/pearl-jam/las

In [ ]:
# To test if there is no song lyric that starts with it:
print(allthesongs["Lyrics"][37])

if (allthesongs["Lyrics"][23]).startswith("sorry, we have no") | (allthesongs["Lyrics"][23]).startswith("we do not have the lyrics for") :
  print ("true") #instead of printing true,

we do not have the lyrics for baby, i'm yours yet.
true


In [ ]:
lyrics = allthesongs["Lyrics"][59]

print(repr(lyrics))
print(lyrics)


stanzas = lyrics.split("\n\r")

print("ONE")
print(repr(stanzas[1]))


"so tired, tired of waiting\r\ntired of waiting for you\r\nso tired, tired of waiting\r\ntired of waiting for you\n\r\ni was a lonely soul\r\ni had nobody till i met you\r\nbut you, keeping me waiting\r\nall of the time, what can i do?\n\r\nit's your life\r\nand you can do what you want\r\ndo what you like\r\nbut please don't keep me waiting\r\nplease don't keep me waiting\n\r\n'cuz i'm so tired, tired of waiting\r\ntired of waiting for you\r\nso tired, tired of waiting\r\ntired of waiting for you\n\r\ni was a lonely soul\r\ni had nobody till i met you\r\nbut you, keeping me waiting\r\nall of the time, what can i do?\n\r\nit's your life\r\nand you can do what you want\r\ndo what you like\r\nbut please don't keep me waiting\r\nplease don't keep me waiting\n\r\n'cuz i'm so tired, tired of waiting\r\ntired of waiting for you\r\nso tired, tired of waiting\r\ntired of waiting for you\r\nfor you, for you"
so tired, tired of waiting
tired of waiting for you
so tired, tired of waiting
tired of

In [ ]:
#Method I created/modified:
def print_all_rhymes_gillian(poem, rhyme_dict, types_perfect, types_imperfect):
    num_perfect = 0
    num_imperfect = 0
    num_not = 0
    real = True
    lines = poem.split('\n')
    #print(lines)
    for index, p in enumerate(lines):
      #print("p:")
      #print(repr(p))
      x
      for index, p in enumerate(lines):
        if p.isspace():
          lines.remove(lines[index])
    rhymes = [verse2rhyme(line, rhyme_dict) for line in lines]
    for i in range(len(lines)):
        for j in range(len(lines)-1):
            if j >= i:
                #print('.', end=' ')
                real == True
            elif rhymes[i][0] == rhymes[j][0]:
                #print("compare:")
                #print(lines[i])
                #print(lines[j])
                num_perfect = num_perfect+1
                if rhymes[i][0] in types_perfect.keys():
                  types_perfect[rhymes[i][0]] = types_perfect[rhymes[i][0]]+1
                else:
                  types_perfect[rhymes[i][0]] = 1
            elif rhymes[i][1] == rhymes[j][1]:
                #print("compare:")
                #print(lines[i])
                #print(lines[j])
                num_imperfect = num_imperfect+1
                if (rhymes[i][0], rhymes[j][0]) in types_imperfect.keys():
                  types_imperfect[(rhymes[i][0], rhymes[j][0])] = types_imperfect[(rhymes[i][0], rhymes[j][0])] +1
                else:
                  types_imperfect[(rhymes[i][0], rhymes[j][0])] = 1
            else:
                #print("compare:")
                #print(lines[i])
                #print(lines[j])
                num_not = num_not+1
    #print("num_perfect: " + str(num_perfect))
    #print("num_imperfect: " + str(num_imperfect))
    #print("num_not: " + str(num_not))
    #print(types_perfect)
    #print(types_imperfect)
    return types_perfect, types_imperfect

def one_song_rhymes(Current_song, types_perfect, types_imperfect):
  punctuation = ['.', ',', ',', ':', ';', '!', '?', ' ', '-', '...', '_']
  stanzas = Current_song.split("\n\n")
  if len(stanzas) ==1:
    stanzas = Current_song.split("\n\r")
  for stanza in stanzas:
    stanza = stanza.replace('[].,,:!? -..._]', '')
    stanza = stanza.replace('\r', '')
    stanza = stanza.replace('"', '')
    print("STANZA")
    print(stanza)
    print(repr(stanza))
    types_perfect, types_imperfect = print_all_rhymes_gillian(stanza, word2rhymes, types_perfect, types_imperfect)
  return types_perfect, types_imperfect
